In [1]:
import torch
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader

# Detect device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Sanity check: detect unique class labels from test set
all_labels = []
for _, labels in test_loader:
    all_labels.extend(labels.tolist())
unique_classes = sorted(set(all_labels))
num_classes = len(unique_classes)
print(f"Unique label values in test set: {unique_classes}")
print(f"Detected number of classes: {num_classes}")

# Load model safely
model = torch.load(
    r"C:\Users\elias\Documents\Neural networks\Advanced\Project\Run1",
    map_location=device
)
model.to(device)
model.eval()

# Initialize confusion matrix
conf_matrix = torch.zeros(num_classes, num_classes, device=device)
total = 0
correct = 0

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating", unit="batch"):
        inputs, labels = batch
        inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=1)

        for t, p in zip(labels, preds):
            # Safety check for bounds
            if t < num_classes and p < num_classes:
                conf_matrix[t.long(), p.long()] += 1
                if t == p:
                    correct += 1
            total += 1

# Calculate overall accuracy
total_accuracy = 100 * correct / total
print(f"\nTotal Accuracy: {total_accuracy:.2f}%\n")

# Convert confusion matrix to % per class
conf_matrix_cpu = conf_matrix.to("cpu")
conf_percent = conf_matrix_cpu.clone()
for i in range(num_classes):
    row_sum = conf_percent[i].sum()
    if row_sum > 0:
        conf_percent[i] = conf_percent[i] / row_sum * 100

# Create and format the confusion DataFrame
columns = [f"Pred_{i}" for i in range(num_classes)]
index = [f"True_{i}" for i in range(num_classes)]
df = pd.DataFrame(conf_percent.numpy(), index=index, columns=columns)
df['% Wrong'] = 100 - df[[f"Pred_{i}" for i in range(num_classes)]].max(axis=1)
df = df.round(2)

# Show results
print("Confusion Matrix (% per true class):")
print(df.to_string())


Using device: cuda


C:\Users\elias\AppData\Local\Temp\ipykernel_15760\1080961450.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(r"C:\Users\elias\Documents\Neural networ

NameError: name 'test_loader' is not defined